In [1]:
import os
import unet
import torch
import dataset
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [2]:
torch.cuda.empty_cache()

In [3]:
loss_list = []
iou_list = []

In [4]:
losslist = []
ioulist = []

In [ ]:
# 训练器
class Trainer:

    def __init__(self, path, model, model_copy, img_save_path):
        self.path = path
        self.model = model
        self.model_copy = model_copy
        self.img_save_path = img_save_path
        # 使用的设备
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 网络
        self.net = unet.UNet().to(self.device)
        # 优化器，这里用的Adam，跑得快点
        self.opt = torch.optim.Adam(self.net.parameters())
        # 这里直接使用二分类交叉熵来训练，效果可能不那么好
        # 可以使用其他损失，比如DiceLoss、FocalLoss之类的
        self.loss_func = nn.BCELoss()
        # 设备好，batch_size和num_workers可以给大点
        self.loader = DataLoader(dataset.Datasets(path), batch_size=1, shuffle=True, num_workers=1)

        # 判断是否存在模型
        if os.path.exists(self.model):
            self.net.load_state_dict(torch.load(model))
            print(f"Loaded{model}!")
        else:
            print("No Param!")
        os.makedirs(img_save_path, exist_ok=True)

    # 训练
    def train(self, stop_value):
        epoch = 1
        while True:
            for inputs, labels in tqdm(self.loader, desc=f"Epoch {epoch}/{stop_value}",
                                       ascii=True, total=len(self.loader)):
                # 将训练使用数据传到设备上
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                # 输出生成的图像
                out = self.net(inputs)
                loss = self.loss_func(out, labels)
                # 后向
                self.opt.zero_grad()
                loss.backward()
                self.opt.step()

                # 输入的图像，取第一张
                x = inputs[0]
                # 生成的图像，取第一张
                x_ = out[0]
                # 标签的图像，取第一张
                y = labels[0]
                # 三张图，从第0轴拼接起来，再保存
                img = torch.stack([x, x_, y], 0)
                intersection = (x_ * y).sum()
                union = x_.sum() + y.sum() - intersection
                IOU = intersection / union
                save_image(img.cpu(), os.path.join(self.img_save_path, f"{epoch}.png"))
                # print("image save successfully !")
            loss_list.append(loss)
            iou_list.append(IOU)
            print(f"\nEpoch: {epoch}/{stop_value}, Loss: {loss}, iou:{IOU}")
            torch.save(self.net.state_dict(), self.model)
            # print("model is saved !")

            epoch += 1


if __name__ == '__main__':
    t = Trainer(r"C:\Users\86177\Desktop\Pytorch-UNet-Retina-master\DRIVE\training", r'./model.plt', r'./model_{}_{}.plt', img_save_path=r'./train_img')
    t.train(200)

No Param!


Epoch 1/200: 100%|#####################################################################| 20/20 [00:11<00:00,  1.67it/s]



Epoch: 1/200, Loss: 0.18924081325531006, iou:0.18171055614948273


Epoch 2/200: 100%|#####################################################################| 20/20 [00:10<00:00,  1.96it/s]



Epoch: 2/200, Loss: 0.17059332132339478, iou:0.258365660905838


...

Epoch 230/200: 100%|###################################################################| 20/20 [00:15<00:00,  1.32it/s]



Epoch: 230/200, Loss: 0.02659926936030388, iou:0.7756372690200806


Epoch 231/200: 100%|###################################################################| 20/20 [00:15<00:00,  1.31it/s]



Epoch: 231/200, Loss: 0.039554938673973083, iou:0.7411676049232483


Epoch 232/200:  80%|#####################################################6             | 16/20 [00:12<00:02,  1.42it/s]

In [ ]:
for i in loss_list:
    loss = i.cpu().detach().numpy()
    losslist.append(loss)
    
for j in iou_list:
    iou = j.cpu().detach().numpy()
    ioulist.append(iou)

In [ ]:
epoch=np.linspace(1, 200, 200)

In [ ]:
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('train loss')
plt.plot(epoch, losslist, 'b-', label='1')

In [ ]:
plt.title('IoU')
plt.xlabel('epoch')
plt.ylabel('train iou')
plt.plot(epoch, ioulist, 'c-', label='1')